In [1]:
pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

  Obtaining dependency information for google-auth from https://files.pythonhosted.org/packages/a1/b1/1272c6e80847ba5349f5ccb7574596393d1e222543f5003cb810865c3575/google_auth-2.40.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-oauthlib from https://files.pythonhosted.org/packages/ac/84/40ee070be95771acd2f4418981edb834979424565c3eec3cd88b6aa09d24/google_auth_oauthlib-1.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2 from https://files.pythonhosted.org/packages/be/8a/fe34d2f3f9470a27b01c9e76226965863f153d5fbe276f83608562e49c04/google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/2d/bd/6aa93c38756cc9fc63262e0dc3d3f1ff7241ce6f413a25ad6e4a9c98b473/google_api_python_client-2.169.0-py3-none-any.whl.metadata
  Obtaining dependency information for requests-oauthlib>=0.7.0 from https://files.pythonhosted.org

In [1]:
from google.oauth2 import credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [2]:
# Step 1: Define Scopes (Permissions you need)
SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly',
          'https://www.googleapis.com/auth/classroom.coursework.students',
          'https://www.googleapis.com/auth/classroom.student-submissions.students.readonly',
          'https://www.googleapis.com/auth/classroom.coursework.me.readonly']

In [3]:
# As Teachers

# Step 2: Authenticate
def authenticate():
    flow = InstalledAppFlow.from_client_secrets_file(
        'client_secret_84676708516-2jfnm72kik0til0ru66r35thkdm3r6tk.apps.googleusercontent.com.json', SCOPES)  # Your downloaded JSON file
    creds = flow.run_local_server(port=0)
    service = build('classroom', 'v1', credentials=creds)
    return service


# Step 3: Fetch Courses
def get_courses(service):
    results = service.courses().list(pageSize=10).execute()
    courses = results.get('courses', [])
    return courses


# Step 4: Fetch Assignments (CourseWork) for a Course
def get_assignments(service, course_id):
    results = service.courses().courseWork().list(courseId=course_id).execute()
    coursework = results.get('courseWork', [])
    return coursework


# Step 5: Fetch Submissions for an Assignment
def get_submissions(service, course_id, coursework_id):
    results = service.courses().courseWork().studentSubmissions().list(
        courseId=course_id,
        courseWorkId=coursework_id
    ).execute()
    submissions = results.get('studentSubmissions', [])
    return submissions

In [10]:
# As Teachers
# Main Flow
if __name__ == '__main__':
    service = authenticate()
    
    print("Fetching Courses...")
    courses = get_courses(service)
    if not courses:
        print('No courses found.')
    else:
        for course in courses:
            print(f"Course: {course['name']} (ID: {course['id']})")
            
            print("\nFetching Assignments...")
            assignments = get_assignments(service, course['id'])
            for assignment in assignments:
                print(f"  Assignment: {assignment['title']} (ID: {assignment['id']})")
                
                print("  Fetching Submissions...")
                submissions = get_submissions(service, course['id'], assignment['id'])
                for submission in submissions:
                    print(f"    Submission ID: {submission['id']}, State: {submission['state']}")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=84676708516-2jfnm72kik0til0ru66r35thkdm3r6tk.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54182%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.courses.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.student-submissions.students.readonly&state=pJL8j8IREgU8JGahyBN268x6cabbfa&access_type=offline
Fetching Courses...
Course: Deep Learning (ID: 762753669189)

Fetching Assignments...


HttpError: <HttpError 403 when requesting https://classroom.googleapis.com/v1/courses/762753669189/courseWork?alt=json returned "The caller does not have permission". Details: "The caller does not have permission">

In [4]:
import os

SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly',
          'https://www.googleapis.com/auth/classroom.student-submissions.me.readonly']

# As Students
def main():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('client_secret_84676708516-2jfnm72kik0til0ru66r35thkdm3r6tk.apps.googleusercontent.com.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('classroom', 'v1', credentials=creds)

    # Step 1: Get your enrolled courses
    courses = service.courses().list().execute().get('courses', [])
    for course in courses:
        print(f"\nCourse: {course['name']} (ID: {course['id']})")

        # Step 2: Get your student submissions (assignments) in this course
        coursework = service.courses().courseWork().list(courseId=course['id']).execute().get('courseWork', [])
        for work in coursework:
            print(f"  Assignment: {work['title']} (ID: {work['id']})")

            submissions = service.courses().courseWork().studentSubmissions().list(
                courseId=course['id'],
                courseWorkId=work['id']
            ).execute().get('studentSubmissions', [])

            for sub in submissions:
                print(f"    Submission state: {sub['state']}")

if __name__ == '__main__':
    main()

NameError: name 'Credentials' is not defined

In [5]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os

# Step 1: Define the required scopes
SCOPES = [
    'https://www.googleapis.com/auth/classroom.courses.readonly',
    'https://www.googleapis.com/auth/classroom.student-submissions.me.readonly'
]

def main():
    creds = None

    # Step 2: Load existing token or initiate OAuth flow
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Step 3: Connect to the Google Classroom API
    service = build('classroom', 'v1', credentials=creds)

    # Step 4: Get your enrolled courses
    results = service.courses().list().execute()
    courses = results.get('courses', [])

    if not courses:
        print("No courses found.")
        return

    for course in courses:
        print(f"\n📘 Course: {course['name']} (ID: {course['id']})")

        # Step 5: Get all coursework for the course
        coursework_list = service.courses().courseWork().list(courseId=course['id']).execute().get('courseWork', [])
        
        for cw in coursework_list:
            print(f"  📄 Assignment: {cw['title']} (ID: {cw['id']})")

            # Step 6: Get your student submission for each assignment
            submissions = service.courses().courseWork().studentSubmissions().list(
                courseId=course['id'],
                courseWorkId=cw['id']
            ).execute().get('studentSubmissions', [])

            for sub in submissions:
                print(f"    📝 Submission State: {sub['state']}")
                if 'assignedGrade' in sub:
                    print(f"    ✅ Grade: {sub['assignedGrade']}")

if __name__ == '__main__':
    main()



📘 Course: Deep Learning (ID: 762753669189)
  📄 Assignment: PCA Assignments (ID: 768915664586)
    📝 Submission State: TURNED_IN
  📄 Assignment: YOLO Assignment (ID: 777466421747)
    📝 Submission State: TURNED_IN
  📄 Assignment: Programming Test (ID: 767247181187)
    📝 Submission State: TURNED_IN
  📄 Assignment: Activation functions  (ID: 762754732251)
    📝 Submission State: TURNED_IN
  📄 Assignment: Perceptron (ID: 762752941568)
    📝 Submission State: TURNED_IN

📘 Course: NLP_cohort1_sem3 (ID: 752624741302)
  📄 Assignment: Midsemester Question paper (ID: 767831396669)
    📝 Submission State: CREATED
  📄 Assignment: Assignment 2 (ID: 771709400852)
    📝 Submission State: TURNED_IN
  📄 Assignment: Assignment 1 (ID: 764631185038)
    📝 Submission State: TURNED_IN

📘 Course: Deep Learning Techniques (ID: 757824498473)

📘 Course: Applied Signal Processing (2024-25) Cohort 1 (ID: 716039841663)
  📄 Assignment: Assignment 5 (ID: 737744237568)
    📝 Submission State: TURNED_IN
  📄 Assignme

In [23]:
# Save into CSV file for NLP pileline

import csv
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Scopes: course list + student submissions
SCOPES = [
    'https://www.googleapis.com/auth/classroom.courses.readonly',
    'https://www.googleapis.com/auth/classroom.student-submissions.me.readonly'
]

def main():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('classroom', 'v1', credentials=creds)

    # Prepare the CSV file
    with open('student_assignments.csv', mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Course Name', 'Course ID', 'Assignment Title', 'Assignment ID', 'Submission State', 'Grade']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        # Step 1: List student courses
        courses = service.courses().list().execute().get('courses', [])

        for course in courses:
            course_id = course['id']
            course_name = course['name']

            # Step 2: List coursework (assignments)
            courseworks = service.courses().courseWork().list(courseId=course_id).execute().get('courseWork', [])

            for cw in courseworks:
                assignment_id = cw['id']
                assignment_title = cw['title']

                # Step 3: Get student's submission status
                submissions = service.courses().courseWork().studentSubmissions().list(
                    courseId=course_id,
                    courseWorkId=assignment_id
                ).execute().get('studentSubmissions', [])

                for sub in submissions:
                    writer.writerow({
                        'Course Name': course_name,
                        'Course ID': course_id,
                        'Assignment Title': assignment_title,
                        'Assignment ID': assignment_id,
                        'Submission State': sub.get('state', 'UNKNOWN'),
                        'Grade': sub.get('assignedGrade', 'Not Graded')
                    })

    print("✅ Data saved to student_assignments.csv")

if __name__ == '__main__':
    main()


✅ Data saved to student_assignments.csv


In [6]:
# Preprocessing & Label Encoding
#| Use Case                  | Description                                         | Target Label Example                          |
#| ------------------------- | --------------------------------------------------- | --------------------------------------------- |
#| Status Classification | Classify if assignment is turned in, returned, etc. | `TURNED_IN`, `RETURNED`, `CREATED`, `MISSING` |
#| Urgency Detection     | Classify if assignment needs urgent attention       | `URGENT`, `ON_TRACK`, `DONE`                  |
#| Grading Status        | Predict if an assignment is graded or not           | `GRADED`, `PENDING`                           |


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Step 1: Load CSV
df = pd.read_csv("student_assignments.csv")

# Step 2: Drop rows with missing titles
df.dropna(subset=['Assignment Title'], inplace=True)

# Step 3: Lowercase & clean assignment titles
df['clean_title'] = df['Assignment Title'].str.lower().str.replace(r'[^a-z0-9\s]', '', regex=True)

# Step 4: Encode submission state as label
le = LabelEncoder()
df['label'] = le.fit_transform(df['Submission State'])

# Save encoder classes for later decoding
label_map = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Encoding Map:", label_map)

# Step 5: Split into train/test
X_train, X_test, y_train, y_test = train_test_split(df['clean_title'], df['label'], test_size=0.2, random_state=42)

# Optional: Save processed data
df[['clean_title', 'label']].to_csv("assignment_nlp_data.csv", index=False)


Label Encoding Map: {'CREATED': 0, 'TURNED_IN': 1}


In [25]:
# NLP Classification Pipeline

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Load the CSV data
df = pd.read_csv("student_assignments.csv")

# Step 2: Basic cleaning
df.dropna(subset=['Assignment Title', 'Submission State'], inplace=True)
df['clean_title'] = df['Assignment Title'].str.lower().str.replace(r'[^a-z0-9\s]', '', regex=True)

# Step 3: Encode submission state labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['Submission State'])

print("🧠 Label encoding map:")
for label, code in zip(le.classes_, le.transform(le.classes_)):
    print(f"  {code}: {label}")

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['clean_title'], df['label'], test_size=0.2, random_state=42)

# Step 5: Text vectorization (TF-IDF)
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Step 6: Train a simple classifier
clf = LogisticRegression(max_iter=200)
clf.fit(X_train_tfidf, y_train)

# Step 7: Evaluate
y_pred = clf.predict(X_test_tfidf)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))
print(f"🎯 Accuracy: {accuracy_score(y_test, y_pred):.2f}")

# Step 8: Sample predictions
def predict_status(title):
    title_clean = title.lower()
    title_clean = ''.join(c for c in title_clean if c.isalnum() or c.isspace())
    vectorized = vectorizer.transform([title_clean])
    pred = clf.predict(vectorized)
    return le.inverse_transform(pred)[0]

print("\n🔍 Sample predictions:")
sample_titles = ["Final Project Submission", "Quiz 2", "Weekly reading", "Assignment 5 draft"]
for title in sample_titles:
    print(f"  '{title}' → {predict_status(title)}")


🧠 Label encoding map:
  0: CREATED
  1: TURNED_IN

📊 Classification Report:
              precision    recall  f1-score   support

     CREATED       0.00      0.00      0.00         1
   TURNED_IN       0.83      1.00      0.91         5

    accuracy                           0.83         6
   macro avg       0.42      0.50      0.45         6
weighted avg       0.69      0.83      0.76         6

🎯 Accuracy: 0.83

🔍 Sample predictions:
  'Final Project Submission' → TURNED_IN
  'Quiz 2' → TURNED_IN
  'Weekly reading' → TURNED_IN
  'Assignment 5 draft' → TURNED_IN


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
